<a href="https://colab.research.google.com/github/sanjanasrinivas22/1BM23CS301-AI/blob/main/ManhattenDistance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
print("Sanjana Srinivas-1BM23CS301")

from heapq import heappush, heappop

# ---------- Problem setup ----------
GOAL = (1, 2, 3,
        8, 0, 4,
        7, 6, 5)  # 0 = blank

# Precompute goal positions
GOAL_POS = {v: (i // 3, i % 3) for i, v in enumerate(GOAL)}

# Heuristic: Manhattan distance
def manhattan_distance(state):
    dist = 0
    for i, v in enumerate(state):
        if v != 0:  # skip blank
            r, c = divmod(i, 3)  # current position (row, col)
            gr, gc = GOAL_POS[v]  # goal position (row, col)
            dist += abs(r - gr) + abs(c - gc)
    return dist

# Inversion parity (checking solvability)
def inversion_parity(state):
    arr = [v for v in state if v != 0]
    inv = 0
    for i in range(len(arr)):
        for j in range(i + 1, len(arr)):
            if arr[i] > arr[j]:
                inv += 1
    return inv % 2

def is_solvable(start, goal):
    """General solvability: start and goal must have same inversion parity."""
    return inversion_parity(start) == inversion_parity(goal)

# Moves: (name, row_delta, col_delta)
MOVES = [
    ("up",    -1,  0),
    ("down",   1,  0),
    ("left",   0, -1),
    ("right",  0,  1),
]

def neighbors(state):
    """Generate (next_state, action) pairs by sliding a tile into the blank."""
    zero = state.index(0)
    zr, zc = divmod(zero, 3)

    for name, dr, dc in MOVES:
        nr, nc = zr + dr, zc + dc
        if 0 <= nr < 3 and 0 <= nc < 3:
            nidx = nr * 3 + nc
            new_state = list(state)
            new_state[zero], new_state[nidx] = new_state[nidx], new_state[zero]
            yield tuple(new_state), name

# ---------- A* search ----------
def a_star(start):
    if not is_solvable(start, GOAL):
        raise ValueError("This puzzle configuration is not solvable.")

    counter = 0
    h0 = manhattan_distance(start)
    frontier = []
    heappush(frontier, (h0, counter, start))

    g = {start: 0}  # g(n): cost to reach the current state from the start
    parent = {start: None}
    action_to = {start: None}

    while frontier:
        f, _, s = heappop(frontier)

        if s == GOAL:
            # Reconstruct path
            actions, states = [], []
            cur = s
            while cur is not None:
                states.append(cur)
                actions.append(action_to[cur])
                cur = parent[cur]
            actions = actions[-2::-1]  # drop None, reverse
            states = states[::-1]
            return actions, states, g

        for ns, act in neighbors(s):
            tentative_g = g[s] + 1  # g(n) = g(parent) + 1
            if ns not in g or tentative_g < g[ns]:
                g[ns] = tentative_g
                parent[ns] = s
                action_to[ns] = act
                counter += 1
                heappush(frontier, (tentative_g + manhattan_distance(ns), counter, ns))

    raise ValueError("No path found (should not happen for solvable states).")

# ---------- Pretty printing ----------
def print_state(state):
    for r in range(3):
        row = state[3*r:3*r+3]
        print(" ".join(str(x) if x != 0 else "·" for x in row))
    print()

# ---------- Example ----------
if __name__ == "__main__":
    start = (2, 8, 3,
             1, 6, 4,
             7, 0, 5)

    print("Start:")
    print_state(start)
    print("Goal:")
    print_state(GOAL)

    try:
        actions, states, g = a_star(start)
        print(f"Solved in {len(actions)} moves using Manhattan distance heuristic.\n")

        # Printing the costs during the search path
        for i, (a, st) in enumerate(zip(actions, states[1:]), start=1):
            g_cost = g[st]  # g(n): cost to reach this state from the start
            h_cost = manhattan_distance(st)  # h(n): Manhattan distance heuristic
            f_cost = g_cost + h_cost  # f(n) = g(n) + h(n)
            print(f"Move {i}: {a}")
            print(f"g(n) = {g_cost}, h(n) = {h_cost}, f(n) = {f_cost}")
            print_state(st)
    except ValueError as e:
        print(e)



Sanjana Srinivas-1BM23CS301
Start:
2 8 3
1 6 4
7 · 5

Goal:
1 2 3
8 · 4
7 6 5

Solved in 5 moves using Manhattan distance heuristic.

Move 1: up
g(n) = 1, h(n) = 4, f(n) = 5
2 8 3
1 · 4
7 6 5

Move 2: up
g(n) = 2, h(n) = 3, f(n) = 5
2 · 3
1 8 4
7 6 5

Move 3: left
g(n) = 3, h(n) = 2, f(n) = 5
· 2 3
1 8 4
7 6 5

Move 4: down
g(n) = 4, h(n) = 1, f(n) = 5
1 2 3
· 8 4
7 6 5

Move 5: right
g(n) = 5, h(n) = 0, f(n) = 5
1 2 3
8 · 4
7 6 5

